# Tesla & GameStop: Stock vs Revenue Dashboard

_Use yfinance and web scraping to build side-by-side price vs revenue dashboards for TSLA and GME._

## Environment & Install
Uncomment the next cell if you need to install packages in your environment.

In [ ]:
# !pip install yfinance pandas plotly beautifulsoup4 lxml requests

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', 50)

In [ ]:
def get_quarterly_revenue_from_macrotrends(ticker, company_slug):
    """Scrape the quarterly revenue table from Macrotrends for a given ticker.
    
    Parameters
    ----------
    ticker : str
        Stock ticker (e.g., 'TSLA')
    company_slug : str
        Lowercase company name used in Macrotrends URL (e.g., 'tesla', 'gamestop')
    Returns
    -------
    DataFrame with columns ['Date', 'Revenue'] sorted by Date ascending.
    """
    url = f"https://www.macrotrends.net/stocks/charts/{ticker}/{company_slug}/revenue"
    # Pull all HTML tables on the page
    tables = pd.read_html(url, flavor='lxml')
    # Heuristic: quarterly revenue table usually has 'Quarterly' in the first column header or a 'Date' and 'Revenue' pair
    qtables = [t for t in tables if t.columns.str.contains('Quarter', case=False).any() or set(['Date','Revenue']).issubset(set(t.columns))]
    if not qtables:
        # fallback: try the first table
        df = tables[0].copy()
    else:
        # choose the widest table (often the quarterly one)
        df = max(qtables, key=lambda t: t.shape[0]).copy()
    # Normalize columns
    cols = [c.strip().replace('\n',' ') for c in df.columns]
    df.columns = cols
    # Rename common variants
    if 'Quarter' in df.columns: df = df.rename(columns={'Quarter':'Date'})
    if 'Revenue (TTM)' in df.columns and 'Revenue' not in df.columns:
        df = df.rename(columns={'Revenue (TTM)':'Revenue'})
    if 'Revenue' not in df.columns:
        # Try to find the first revenue-like column
        rev_col = next((c for c in df.columns if 'Revenue' in c), None)
        if rev_col: df = df.rename(columns={rev_col:'Revenue'})
    # Keep Date + Revenue
    keep = [c for c in ['Date','Revenue'] if c in df.columns]
    df = df[keep].copy()
    # Clean Revenue: remove $ and commas; coerce to float (Millions of USD on Macrotrends)
    df['Revenue'] = (df['Revenue'].astype(str)
                     .str.replace(r'\$|,', '', regex=True)
                     .replace({'': np.nan, 'None': np.nan})).astype(float)
    # Drop missing and sort
    df = df.dropna(subset=['Revenue']).drop_duplicates(subset=['Date']).sort_values('Date')
    df.reset_index(drop=True, inplace=True)
    return df

In [ ]:
def make_graph(stock_data, revenue_data, title):
    """Create a two-panel Plotly figure with Close price (left) and Revenue (right)."""
    fig = make_subplots(rows=1, cols=2, subplot_titles=(f"{title} — Close Price", f"{title} — Quarterly Revenue (Millions USD)"))
    
    # Left: closing prices
    fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close'], name='Close', mode='lines'), row=1, col=1)
    
    # Right: revenue bar
    fig.add_trace(go.Bar(x=pd.to_datetime(revenue_data['Date']), y=revenue_data['Revenue'], name='Revenue (M)'), row=1, col=2)
    
    fig.update_xaxes(title_text='Date', row=1, col=1)
    fig.update_yaxes(title_text='Price (USD)', row=1, col=1)
    fig.update_xaxes(title_text='Quarter', row=1, col=2)
    fig.update_yaxes(title_text='Revenue (M USD)', row=1, col=2)
    fig.update_layout(title_text=title, showlegend=False, height=500, width=1000)
    return fig

## Question 1 — Extract Tesla Stock Data Using `yfinance`

In [ ]:
tesla_ticker = yf.Ticker('TSLA')
tesla_data = tesla_ticker.history(period='max')
tesla_data.reset_index(inplace=True)
tesla_data.head()

## Question 2 — Extract Tesla Revenue via Web Scraping

In [ ]:
tesla_revenue = get_quarterly_revenue_from_macrotrends('TSLA', 'tesla')
tesla_revenue.tail()

## Question 3 — Extract GameStop Stock Data Using `yfinance`

In [ ]:
gme_ticker = yf.Ticker('GME')
gme_data = gme_ticker.history(period='max')
gme_data.reset_index(inplace=True)
gme_data.head()

## Question 4 — Extract GameStop Revenue via Web Scraping

In [ ]:
gme_revenue = get_quarterly_revenue_from_macrotrends('GME', 'gamestop')
gme_revenue.tail()

## Question 5 — Tesla Stock and Revenue Dashboard

In [ ]:
fig_tsla = make_graph(tesla_data.set_index('Date'), tesla_revenue, 'Tesla (TSLA)')
fig_tsla.show()

## Question 6 — GameStop Stock and Revenue Dashboard

In [ ]:
fig_gme = make_graph(gme_data.set_index('Date'), gme_revenue, 'GameStop (GME)')
fig_gme.show()

## Question 7 — Share your Assignment Notebook
1. Save the notebook as `tsla_gme_dashboard.ipynb`.
2. Push it to a **public GitHub repo** (Add file → Upload files → Commit).
3. Paste the GitHub link in your assignment submission.
